In [9]:
pwd

'/users/RONDI/project'

In [10]:
cd /users/RONDI/project

/users/RONDI/project


In [11]:
pwd

'/users/RONDI/project'

In [12]:
# wget https://www.encodeproject.org/files/ENCFF168GAN/@@download/ENCFF168GAN.bed.gz
# wget https://www.encodeproject.org/files/ENCFF027BPY/@@download/ENCFF027BPY.bed.gz
# wget https://www.encodeproject.org/files/ENCFF886GET/@@download/ENCFF886GET.bed.gz
# wget https://www.encodeproject.org/files/ENCFF931FYG/@@download/ENCFF931FYG.bed.gz
# wget https://www.encodeproject.org/files/ENCFF448LQD/@@download/ENCFF448LQD.bed.gz
# wget https://www.encodeproject.org/files/ENCFF027ZYL/@@download/ENCFF027ZYL.bed.gz
# wget https://www.encodeproject.org/files/ENCFF016INZ/@@download/ENCFF016INZ.bed.gz
# wget https://www.encodeproject.org/files/ENCFF660AEY/@@download/ENCFF660AEY.bed.gz
# wget https://www.encodeproject.org/files/ENCFF478ZBE/@@download/ENCFF478ZBE.bed.gz
# wget https://www.encodeproject.org/files/ENCFF257BSS/@@download/ENCFF257BSS.bed.gz

# gunzip ENCFF016INZ.bed.gz  ENCFF027BPY.bed.gz  ENCFF027ZYL.bed.gz  ENCFF168GAN.bed.gz  ENCFF214BTG.bed.gz  ENCFF478ZBE.bed.gz  ENCFF660AEY.bed.gz  ENCFF824LVP.bed.gz  ENCFF841PMG.bed.gz  ENCFF931FYG.bed.gz

# (base) RONDI@node0:/proj/SIUE-CS590-490/reference$ cp /proj/SIUE-CS590-490/reference/* ~/project

# (base) RONDI@node0:~/project$ reference_genome="hg38.fa"
# output_file="atgc_combinedinput.txt"
# > "$output_file"
# for bed_file in *.bed; do
#     bedtools getfasta -fi "$reference_genome" -bed "$bed_file" >> "$output_file"
# done
# echo "All sequences have been saved to $output_file."
# All sequences have been saved to atgc_combinedinput.txt.

# bedtools getfasta -fi hg38.fa -bed ENCFF016INZ.bed -fo ENCFF016INZ.txt



In [13]:
#FINAL POSTIVE FILE EXTRACTIONS:

# check if the diff is >= than average_difference and := extract all those rows and >: extarct first average_difference length sequences into postive_nucleotides.
import math

input_file = "ENCFF016INZ.txt"  # Replace with your actual input file name
output_file = "chromosome_extracted_data.txt"
trimmed_output_file = "postive_nucleotides.txt"

# List to store extracted data and differences
extracted_data = []
differences = []

# chromosome_extracted_data.txt" 778747L, 194469836B                                                                                                                                       2,1           Top
# Extracted data has been saved to chromosome_extracted_data.txt.
# The average difference is 221.00
# Trimmed sequences have been saved to postive_nucleotides.txt.

# Open the input file and process each line
with open(input_file, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            # Extract chromosome, start, and end positions
            header_parts = line[1:].split(":")  # Remove '>' and split by ':'
            chromosome = header_parts[0]
            start_end = header_parts[1].split("-")
            start = int(start_end[0])
            end = int(start_end[1])
            if start < end:
                difference = end - start  # Calculate the difference
                sequence = next(file).strip()
                # Append to data list
                extracted_data.append((chromosome, start, end, difference, sequence))
                differences.append(difference)

# Write the extracted data to the output file
with open(output_file, 'w') as outfile:
    # Header
    outfile.write("Chromosome\tStart\tEnd\tDifference\tSequence\n")
    # Data rows
    for chromosome, start, end, difference, sequence in extracted_data:
        outfile.write(f"{chromosome}\t{start}\t{end}\t{difference}\t{sequence}\n")

print(f"Extracted data has been saved to {output_file}.")

# Calculate the average difference
if differences:
    average_difference = math.ceil(sum(differences) / len(differences))
    print(f"The average difference is {average_difference:.2f}")


    # Write the trimmed data to a new output file
    with open(trimmed_output_file, 'w') as trimmed_outfile:
        # trimmed_outfile.write("Sequence\n")
        for chromosome, start, end, difference, sequence in extracted_data:
            if difference >= average_difference:
                # Trim sequence to only the first 'average_difference' length
                trimmed_sequence = sequence[:average_difference]
                
                
                # Write the trimmed sequence directly to the output file
                trimmed_outfile.write(f"\n{trimmed_sequence}\n")

    print(f"Trimmed sequences have been saved to {trimmed_output_file}.")

else:
    print("No valid chromosome data found to calculate the average difference.")



Extracted data has been saved to chromosome_extracted_data.txt.
The average difference is 223.00
Trimmed sequences have been saved to postive_nucleotides.txt.


In [14]:
#extracting negative file
# calculate negative file end of row1-start of row2 > avgdiff craete in .bed file new row as row1-start+1 and row1-start + avg 
import math

input_file = "chromosome_extracted_data.txt"
output_bed_file = "negative_file.bed"

# Define the average difference

# List to store data from input file
data = []

# Read the data from the input file
with open(input_file, 'r') as file:
    next(file)  # Skip header line
    for line in file:
        # Parse line into respective components
        line_data = line.strip().split("\t")
        chromosome = line_data[0]
        start = int(line_data[1])
        end = int(line_data[2])
        difference = int(line_data[3])
        sequence = line_data[4]
        data.append((chromosome, start, end, difference, sequence))

# Process each pair of rows to check the gap and create new entries as necessary
with open(output_bed_file, 'w') as outfile:
    for i in range(len(data) - 1):
        current_chromosome, current_start, current_end, _, _ = data[i]
        next_chromosome, next_start, _, _, _ = data[i + 1]

        if current_chromosome == next_chromosome:
            gap_difference = next_start - current_end

            # Check if the gap is equal or greater than the average difference
            if gap_difference == average_difference:
                outfile.write(f"{current_chromosome}\t{new_start}\t{new_end}\n")

            elif gap_difference > average_difference:
                new_start = current_end + 1
                new_end = new_start + average_difference

                # Write the new entry to the output file
                outfile.write(f"{current_chromosome}\t{new_start}\t{new_end}\n")
            else: 
                pass
print(f"New BED entries have been saved to {output_bed_file}.")


New BED entries have been saved to negative_file.bed.


In [ ]:
# (base) RONDI@node0:~/project$ bedtools getfasta -fi hg38.fa -bed negative_file.bed -fo negative_sequence.txt
# bedtools getfasta -fi hg38.fa -bed negative_file.bed -fo negative_sequence.txt

In [15]:
#after sending above .txt to bed tools extracting only all -ve squence s to one file:
# Define the input and output file paths
input_file = "negative_sequence.txt"  # Replace with your actual file name containing the sequences
output_file = "final_negative.txt"

# Open the input file and process each line to extract sequences
with open(input_file, 'r') as file, open(output_file, 'w') as outfile:
    for line in file:
        line = line.strip()
        if not line.startswith(">"):  # Ignore header lines starting with '>'
            # print(len(line))
            outfile.write("\n"+ line + "\n")  # Write each sequence on a new line

print(f"All sequences have been saved to {output_file}.")


All sequences have been saved to final_negative.txt.
